In [1]:
import torch
from torch import nn

import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

### data

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),  # 원래 이미지는 HxWxC 이지만, ToTensor를 사용하면 CxHxW로 변환된다.
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))] # 3개의 채널에 대해 각각 mean, std를 설정해준다. 0~255 -> -1~1 0.5를 기준으로 정규분포
)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=None)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=None)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

Files already downloaded and verified
Files already downloaded and verified


In [6]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
         "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
         "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"] }

In [7]:
class VGG(nn.Module):
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) # 7x7 이 되도록 avg pooling 하는 녀석
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg:
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]
        return nn.Sequential(*layers)

In [4]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
print(avgpool(torch.randn(2,3,2,2))) # 작은 놈이 들어오면 늘려서라도 맞춰준다
# print(avgpool(torch.randn(2,3,1,1))) # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[-1.0738, -1.0738,  0.5207,  0.5207],
          [-1.0738, -1.0738,  0.5207,  0.5207],
          [-0.6337, -0.6337, -2.3645, -2.3645],
          [-0.6337, -0.6337, -2.3645, -2.3645]],

         [[ 0.1368,  0.1368, -0.1892, -0.1892],
          [ 0.1368,  0.1368, -0.1892, -0.1892],
          [-0.7608, -0.7608, -0.8586, -0.8586],
          [-0.7608, -0.7608, -0.8586, -0.8586]],

         [[ 0.6589,  0.6589, -0.3580, -0.3580],
          [ 0.6589,  0.6589, -0.3580, -0.3580],
          [ 0.9405,  0.9405, -0.5415, -0.5415],
          [ 0.9405,  0.9405, -0.5415, -0.5415]]],


        [[[ 1.3030,  1.3030,  0.8989,  0.8989],
          [ 1.3030,  1.3030,  0.8989,  0.8989],
          [-0.6455, -0.6455,  0.2551,  0.2551],
          [-0.6455, -0.6455,  0.2551,  0.2551]],

         [[-1.2722, -1.2722, -1.0349, -1.0349],
          [-1.2722, -1.2722, -1.0349, -1.0349],
          [-0.8495, -0.8495,  0.4909,  0.4909],
          [-0.8495, -0.8495,  0.4909,  0.4909]],

   

In [5]:
model = VGG(cfgs["E"], batch_norm=False)
# print(model)
! pip install torchinfo
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [2, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [2, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [2, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [2, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [2, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [2, 256, 56, 56]          29

In [6]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


### Train

for 